# Hierarchical RNN for dialogue
The baseline hierarchical model from "Hierarchical Text Generation and Planning for Strategic Dialogue."

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
import random
import json
from tensorflow.python.layers.core import Dense

In [4]:
import sys
sys.path.append("../src/models/")
sys.path.append('../src/models/agents/')
sys.path.append('../src/data/')
from agent import Agent
from parse import SentenceParser

In [5]:
train_iterations = 300
learning_rate = 0.1
max_input_length = 6
max_output_length = 20
unk_threshold = 20

### Sentence-level parsing
Not dealing with the final action data for now. Training examples are lists of utterances.

In [6]:
parser = SentenceParser(unk_threshold=unk_threshold,
                  input_directory="../data/raw/",
                  output_directory="../data/tmp/")
print("Vocab size: {}".format(parser.vocab_size))

Vocab size: 502


In [7]:
parser.parse()

### Hierarchical Agent

In [8]:
class HierarchicalAgent(Agent):    
    def _init_placeholders(self):
        self.encoder_inputs = tf.placeholder(
            shape=[None, None, None],
            dtype=tf.int32,
            name="encoder_inputs")

        self.encoder_lengths = tf.placeholder(
            shape=[None, None],
            dtype=tf.int32,
            name="encoder_lengths")

        self.decoder_inputs= tf.placeholder(
            shape=[None, None, None],
            dtype=tf.int32,
            name="decoder_inputs")

        self.decoder_targets = tf.placeholder(
            shape=[None, None, None],
            dtype=tf.int32,
            name="decoder_targets")

        self.decoder_lengths = tf.placeholder(
            shape=[None, None],
            dtype=tf.int32,
            name="decoder_lengths")
        
    def encoding_layer(self):
        self.encoder_cell = tf.contrib.rnn.MultiRNNCell([
            tf.nn.rnn_cell.LSTMCell(
                self.hidden_dim, 
                initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=1))
            for _ in range(self.num_layers)])
        
        self.encoder_final_states = tf.map_fn(encode_step, self.embedded_encoder_inputs)
    
    def encode_step(self, embedded_encoder_inputs):
        encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(
            cell=self.encoder_cell,
            inputs=embedded_encoder_inputs,
            sequence_length=self.encoder_lengths,
            dtype=tf.float32,
            scope="encoding_layer")
        return encoder_final_state
        
    def context_layer(self):
        self.context_cell = tf.contrib.rnn.MultiRNNCell([
            tf.nn.rnn_cell.LSTMCell(
                self.hidden_dim, 
                initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            for _ in range(self.num_layers)])
        
        # Edit sequence length:
        context_outputs, context_final_state = tf.nn.dynamic_rnn(
            cell=self.context_cell,
            inputs=self.encoder_final_states,
            sequence_length=self.encoder_lengths,
            dtype=tf.float32,
            scope="context_layer")
        
    def decoding_layer(self):
        
    
    def decoding_training(self):
        self.decoder_cell = tf.contrib.rnn.MultiRNNCell([
            tf.nn.rnn_cell.LSTMCell(
            self.hidden_dim, 
            initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2)) for _ in range(self.num_layers)])

        self.output_layer = Dense(
            units=self.vocab_size,
            kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))

        training_helper = tf.contrib.seq2seq.TrainingHelper(
            inputs=self.embedded_decoder_inputs,
            sequence_length=self.decoder_lengths,
            time_major=False)
        
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
            cell=self.decoder_cell,
            helper=training_helper,
            initial_state=self.encoder_final_state,
            output_layer=self.output_layer)

        training_outputs = tf.contrib.seq2seq.dynamic_decode(
            decoder=training_decoder,
            impute_finished=True,
            maximum_iterations=self.max_output_length)[0]
        
    
    def build_graph(self):
        self._init_placeholders()
        self._define_embedding()
        self.encoding_layer()
        self.context_layer()
        self.decoding_layer()

In [9]:
a = HierarchicalAgent(vocab=parser.vocab,
              max_iter=train_iterations,
              eta=learning_rate,
              max_input_length=max_input_length,
              max_output_length=max_output_length,
              hidden_dim=64)

In [62]:
a._init_placeholders()

In [63]:
a._define_embedding()